In [10]:
import pandas as pd
from sklearn import preprocessing

In [2]:
all_features = ["tweet_id", "tweet_timestamp", "engaged_user_id", "engaging_user_id", "retweet_with_comment"]
train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [2, 8, 9, 14, 22], sep="\x01"
                   )

In [3]:
train['retweet_with_comment_bool'] = train.retweet_with_comment.fillna(0)
train.loc[train.retweet_with_comment_bool != 0.0, 'retweet_with_comment_bool'] = 1.0

In [4]:
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()

In [5]:
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100)))

In [6]:
train_set = train_set[['engaged_user_id', 'engaging_user_id', 'retweet_with_comment_bool']]
val_set = val_set[['engaged_user_id', 'engaging_user_id', 'retweet_with_comment_bool']]

In [7]:
pos_train_set = train_set[train_set['retweet_with_comment_bool'] == 1.0]
num_times_engaging_user_did_action = pos_train_set.groupby(['engaging_user_id']).size().reset_index(name = 'count').sort_values(ascending=False, by = 'count')

NameError: name 'positive_training_subset_columns' is not defined

In [8]:
num_times_engaging_user_did_action = pos_train_set.groupby(['engaging_user_id']).size().reset_index(name = 'count').sort_values(ascending=False, by = 'count')

In [11]:
x_df = num_times_engaging_user_did_action.drop(['engaging_user_id'], axis = 1)
x = x_df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

In [12]:
df

,0
0,1.000000
1,0.941176
2,0.921569
3,0.921569
4,0.803922
...,...
877408,0.000000
877409,0.000000
877410,0.000000
877411,0.000000


In [13]:
num_times_engaging_user_did_action_scaled = pd.concat([num_times_engaging_user_did_action, df], axis =1)

In [21]:
num_times_engaging_user_did_action

,engaging_user_id,count
821311,EFB915DFC2D39C665F88ACC1353768F5,52
199520,3A50CD1BAD83B11D7CFE10A36B65EEA2,49
458475,85AB0031C794E8B0D81074EADB2D81BD,48
547220,9F99B044C0EBF9C53BDF83FD7D8BC149,48
637756,BA1070B009B94F11EE528B28817BFE69,42
...,...,...
311194,5ACA2795F3A1AAE8B75CB038562C48D4,1
311195,5ACA307B5CA1ED755A6081C296353F23,1
311196,5ACA428CA1E335F55313E11D0B559FB5,1
311197,5ACA53E42959A05F524D87AB3C7C7FCA,1


In [22]:
df

,0
0,1.000000
1,0.941176
2,0.921569
3,0.921569
4,0.803922
...,...
877408,0.000000
877409,0.000000
877410,0.000000
877411,0.000000


In [20]:
num_times_engaging_user_did_action_scaled

,engaging_user_id,count,0
0,00000B85AAF7DE172876FD96718C4469,2,1.000000
1,000038E85ED799B9F6E29992ABB7952D,1,0.941176
2,000070F30341E475432BAAA845117D6C,2,0.921569
3,00008068CE06231116E5C6E95769ABBB,1,0.921569
4,000088971D76A0C34F5D5D4246E0E433,1,0.803922
...,...,...,...
877408,FFFFABF1EF4A5E4D74A74F010B656FED,1,0.000000
877409,FFFFB5252BC3E9F5AAC0ABF28528EF0C,2,0.000000
877410,FFFFBC5094776E13497B5A03AD6A30B5,1,0.000000
877411,FFFFC0C8E7CB8510BF17D33CAB541C90,2,0.000000


In [14]:
train_set_action = pd.merge(train_set, num_times_engaging_user_did_action_scaled, how = 'left', on = 'engaging_user_id')
val_set_action = pd.merge(val_set, num_times_engaging_user_did_action_scaled, how = 'left', on = 'engaging_user_id')

In [26]:
train_set_action

,engaged_user_id,engaging_user_id,retweet_with_comment_bool,count,0
0,B2263AF9B63CBC88D666E34277F567A3,12A1AF0088C5B4FCBFA024D4A1971323,0.0,0.0,0.000000
1,E714521889A38AF35386AA7F7BFC01EF,C836F432B0FC7847C2C753949C51B961,0.0,0.0,0.000000
2,A28FC60D0555E0BBDF7A5717BAE41DBD,77309DE08AB631D8224254D33648F287,0.0,0.0,0.000000
3,72CB45172B3B726002426A9FE18D2061,91363F238C79DAEE42E2A2C97A5F8E7C,0.0,0.0,0.000000
4,443703CB3D0505C9CAFC97D27B8FB2E7,EFA4C5B62E097EB203F8AFDC470AEB27,1.0,1.0,0.000000
...,...,...,...,...,...
133267709,D75AAE3CCFA44BD84C35942CA4D03E56,60B93589A97D9DB7023A4BB460887168,0.0,0.0,0.000000
133267710,D75AAE3CCFA44BD84C35942CA4D03E56,ACD8BE62B45E077B63D1EB6C5CF8A5F3,0.0,3.0,0.000000
133267711,D856EFF41651280507E4167650C679C5,6F9421EE4CEC588974725CF5A811CCA5,0.0,0.0,0.000000
133267712,D75AAE3CCFA44BD84C35942CA4D03E56,1195831650BF6FB0B0C11245C97E9227,1.0,1.0,0.019608


In [25]:
val_set_action

,engaged_user_id,engaging_user_id,retweet_with_comment_bool,count,0
0,D75AAE3CCFA44BD84C35942CA4D03E56,A4B1828B571271073C68DA93D1465484,0.0,1.0,0.0
1,C7677DF4D001A22F3B969B2ABAB1C179,F3A4C8EB81ADF3685E8F693037808A2C,0.0,0.0,0.0
2,46AA727F36DE4EC3CAD7AC2C312421C3,8BD011AB59AC04491687749245009C60,0.0,0.0,0.0
3,258600B339EB2CBB1006073A145B9CFC,F7BFBA56274FCBBBB76BDD36D0D6495B,0.0,0.0,0.0
4,3CE0AF4B64BC1FA67EFA4E98B38CDF08,10F6E035959B9AC156CED11E0FAED566,0.0,0.0,0.0
...,...,...,...,...,...
14807518,63A88B9A7264F35CBFE2C9DFFB428F36,A18266F8C443499B212E666BD19536B7,0.0,0.0,0.0
14807519,BCDC7E16708AC5B155BA52460DFF687F,159C36A1D94F654C5BC48F40D0D76AF9,0.0,0.0,0.0
14807520,63A88B9A7264F35CBFE2C9DFFB428F36,053D77B11E4B8D6A514715BDA396295D,0.0,0.0,0.0
14807521,63A88B9A7264F35CBFE2C9DFFB428F36,BC210F06D8397A7C4CD396152136F10C,0.0,0.0,0.0


In [27]:
(val_set_action[0] != 0.0).sum()

83879

In [28]:
(val_set_action[0] == 0.0).sum()

14723644

In [24]:
train_set_action[0].fillna(0.0, inplace = True)
val_set_action[0].fillna(0.0, inplace = True)

In [30]:
train_set_action.drop(['count'], axis = 1).to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f2/train_set.csv', sep = '\t', index = False)

In [31]:
val_set_action.drop(['count'], axis = 1).to_csv('/home/ubuntu/recsys_challenge_2020/classification_type_models/f2/val_set.csv', sep = '\t', index = False)

In [32]:
train_set_action

,engaged_user_id,engaging_user_id,retweet_with_comment_bool,count,0
0,B2263AF9B63CBC88D666E34277F567A3,12A1AF0088C5B4FCBFA024D4A1971323,0.0,0.0,0.000000
1,E714521889A38AF35386AA7F7BFC01EF,C836F432B0FC7847C2C753949C51B961,0.0,0.0,0.000000
2,A28FC60D0555E0BBDF7A5717BAE41DBD,77309DE08AB631D8224254D33648F287,0.0,0.0,0.000000
3,72CB45172B3B726002426A9FE18D2061,91363F238C79DAEE42E2A2C97A5F8E7C,0.0,0.0,0.000000
4,443703CB3D0505C9CAFC97D27B8FB2E7,EFA4C5B62E097EB203F8AFDC470AEB27,1.0,1.0,0.000000
...,...,...,...,...,...
133267709,D75AAE3CCFA44BD84C35942CA4D03E56,60B93589A97D9DB7023A4BB460887168,0.0,0.0,0.000000
133267710,D75AAE3CCFA44BD84C35942CA4D03E56,ACD8BE62B45E077B63D1EB6C5CF8A5F3,0.0,3.0,0.000000
133267711,D856EFF41651280507E4167650C679C5,6F9421EE4CEC588974725CF5A811CCA5,0.0,0.0,0.000000
133267712,D75AAE3CCFA44BD84C35942CA4D03E56,1195831650BF6FB0B0C11245C97E9227,1.0,1.0,0.019608


In [ ]:
train_set_action

In [33]:
data_one_hot_sparse = pd.get_dummies(train_set_action, columns=['engaged_user_id', 'engaging_user_id'], sparse=True)

KeyboardInterrupt: 